# Загрузка базы вопросов ЧГК

Нужно загрузить все доступные вопросы и ответы к ним с базы http:///db.chgk.info . Результаты необходимо сохранить в каком-либо формате в форме вопрос-ответ (pickle, sqlite, текстовый файл и т.д.). 

In [59]:
from bs4 import BeautifulSoup
import requests

In [60]:
path = 'http://db.chgk.info'
r = requests.get('http://db.chgk.info')

In [61]:
soup = BeautifulSoup(r.text, 'lxml')

In [109]:
import re
last_page = soup.find('a', title='Перейти на последнюю страницу')['href']
n_of_pages = int(re.compile('\d+').search(last_page).group(0))

In [279]:
import urllib.request

def get_image(img_tag):
    """
    gets img_tag and saves files to Data/images_chgk/
    
    returns file name 
            or None if there is no image
            
    example: 123123.jpg
    """
    
#     regular expression for image name
    reg = re.compile('[\d]+.[\w]+$')
    if img_tag:
#         img_tag usualy looks like '<img src="http://db.chgk.info/images/db/20160407.jpg">'
        img_url = img_tag['src']
        img_name_found = reg.search(img_url)
        if img_name_found:
            img_name = img_name_found.group(0)
        else:
            return None
        urllib.request.urlretrieve(img_url, 'Data/images_chgk/' + img_name)
        return img_name
    return None

In [254]:
def get_question(block):
    """
    gets block -- <div class = "question">
    
    returns tuple (question_text, image_name) where:
        question_text -- all text found in before <div class="collapsible"> including 'razdatka'
        image_names -- name of image found before <div class="collapsible">. It is saved in 'Data/images_chgk/'.
    """

    question_text = ''

    for t in block.find_all(text=True, recursive=False):
        if t: question_text += ' ' + t.strip()
    
    for t in block.find('p').find_all(text=True, recursive=False):
        if t: question_text += ' ' + t.strip()
    
    razdatka = block.find('div', 'razdatka')
    if razdatka:
        r_texts = razdatka.find_all(text=True, recursive=True)
        for t in r_texts:
            question_text += ' ' + t.strip()
    
#     get_image returns None if there is no image
    give_out = block.find('img')
    image_name = get_image(give_out)

    return question_text, image_name

In [255]:
def get_answer(block):
    """
    block -- <div class="collapse-processed">
    
    returns tuple (answer_text, image_name, comment_text)
    """
    answer_block = block.find('p')
    answer_text = answer_block.get_text().strip()
    
    comment_block = answer_block.find_next_sibling('p')
    if comment_block:
        comment_text = comment_block.get_text().strip()
    else:
        comment_text = None
    image_name = get_image(block.find('img'))
    
    return answer_text, image_name, comment_text
    

In [256]:
def get_questions(page):
    """
    page -- page on db.chgk.info with questions
    
    returns list of tuple where each tuple: 
        (question text;
        image name for question if there is one. if not, it is None;
        answer text;
        image name for answer if there is one. if not, it is None;
        comment text if there is one. in not, it is None.)
    """
    questions = []
    soup_quest = BeautifulSoup(page.text, 'lxml')
    for s, a in zip(soup_quest.find_all('div', 'question'), soup_quest.find_all('div', 'collapsible')):
        question_text, q_image_name = get_question(s)
        answer_text, ans_image_name, comment_text = get_answer(a)
        
        if question_text == '':
            print(page.url)
        else:
            questions.append((question_text, q_image_name, answer_text, ans_image_name, comment_text))
    return questions

In [261]:
def get_links(page):
    """
    page -- html that was got from iterating over links on main page
    
    returns pack of links to pages with questions
    """
    soup_pack = BeautifulSoup(page.text, 'lxml')
    pack = []
    for o in soup_pack.find_all('tr', 'odd'):
        pack.append(path + o.find('a')['href'])
    for e in soup_pack.find_all('tr', 'even'):
        pack.append(path + e.find('a')['href'])
    return pack

In [ ]:
questions = []
for i in range(n_of_pages+1):
    print(i)
    payload = {'page' : i}
    page = requests.get('http://db.chgk.info/last', params=payload)
    links = get_links(page)
    for l in links:
        q = requests.get(l)
        quests = get_questions(q)
        questions.extend(quests)

In [285]:
questions[:5]

[('     Согласно порталу WomanAdvice, в кинокомпании "Sony" боятся провала\nочередного продолжения франшизы, поэтому сделали предложение известному\nчеловеку получить 15 долларов за 7. Какие девять символов мы пропустили\nв тексте вопроса?',
  None,
  'Ответ: 000000000.',
  None,
  'Зачёт: Девять нолей.'),
 ('     Английский король Генрих VII Тюдор, выросший за рубежом, вернулся на\nродину с войском наемников, намереваясь захватить власть. Под флагом\nкакого княжества сражались войска Генриха?',
  None,
  'Ответ: Уэльс.',
  None,
  'Зачёт: Уэльского.'),
 ('     Психолог Бен Амбридж отмечает, что женщины отличаются от мужчин не\nстолько способностями, сколько более сложным внутренним миром. Объясняя\nэти различия, он приводит в пример не Венеру, а икс. Какие четыре буквы\nмы пропустили в предыдущем предложении?',
  None,
  'Ответ: С, н, е, р.',
  None,
  'Комментарий: По мнению психолога, мужчины и женщины — это не Марс и Венера, а,\nскорее, "Марс" и "Сникерс": в общем-то, одинаковые, т

In [282]:
import pickle
with open('Data/pickles/questions_chgk.pickle', 'wb') as f:
    pickle.dump(questions, f)

In [283]:
q_from_pickle = None
with open('Data/pickles/questions_chgk.pickle', 'rb') as f:
    q_from_pickle = pickle.load(f)

In [284]:
q_from_pickle[:5]

[('     Согласно порталу WomanAdvice, в кинокомпании "Sony" боятся провала\nочередного продолжения франшизы, поэтому сделали предложение известному\nчеловеку получить 15 долларов за 7. Какие девять символов мы пропустили\nв тексте вопроса?',
  None,
  'Ответ: 000000000.',
  None,
  'Зачёт: Девять нолей.'),
 ('     Английский король Генрих VII Тюдор, выросший за рубежом, вернулся на\nродину с войском наемников, намереваясь захватить власть. Под флагом\nкакого княжества сражались войска Генриха?',
  None,
  'Ответ: Уэльс.',
  None,
  'Зачёт: Уэльского.'),
 ('     Психолог Бен Амбридж отмечает, что женщины отличаются от мужчин не\nстолько способностями, сколько более сложным внутренним миром. Объясняя\nэти различия, он приводит в пример не Венеру, а икс. Какие четыре буквы\nмы пропустили в предыдущем предложении?',
  None,
  'Ответ: С, н, е, р.',
  None,
  'Комментарий: По мнению психолога, мужчины и женщины — это не Марс и Венера, а,\nскорее, "Марс" и "Сникерс": в общем-то, одинаковые, т